In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**IMPORT** test and train datasets

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e2/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e2/test.csv')

In [ ]:
train.head()

**PREPROSSING**

In [ ]:
train.drop(['work_type','avg_glucose_level'],axis=1,inplace=True)
test.drop(['work_type','avg_glucose_level'],axis=1,inplace=True)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
col = ['ever_married','Residence_type','smoking_status','gender']
for i in col:
    train[i]=le.fit_transform(train[i])
    test[i]=le.transform(test[i])
train.head()

**TRAIN_TEST_SPLIT** for training

In [ ]:
x = train.drop(['stroke'], axis = 1)
y = train['stroke']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.2, random_state=1)

**MODEL TRAINING**

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(x_train,y_train)

**PREDICTIONS AND ACCURACY SCORE**

In [ ]:
predictions = classifier.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)

In [ ]:
accuracy

**FINE TUNING THE MODEL**

In [ ]:
param_grid = {
    'penalty' : ['l1','l2','elasticnet','None'],
    'C': np.logspace(-4,4,20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100,150,200,250,300,350]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
GS = GridSearchCV(
    estimator = classifier,
    param_grid = param_grid,
    cv = 3,
    n_jobs = -1,
)

In [ ]:
GS.fit(x_train,y_train)

**Best Paramaters for the model**

In [ ]:
print(GS.best_params_)

In [ ]:
print(GS.best_score_)

**ACCURACY AFTER FINE TUNING THE PARAMATERS **
> which is 0.00032669062 more than before 

In [ ]:
tunedpredictions = GS.predict(x_test)
accuracy = accuracy_score(y_test, tunedpredictions)
accuracy

In [ ]:
submission_prediction = GS.predict(test)
idss = test["id"]

In [ ]:
ddf = pd.DataFrame({"id":idss.values,
                   "stroke":submission_prediction})

In [ ]:
ddf.to_csv("submissio.csv", index = False)